### RAG Chain

In [1]:
sample_docs = [
  """Deep learning is a subfield of machine learning that uses neural networks with multiple layers. It is widely applied in image recognition, natural language processing, and speech recognition.""",

  """Machine learning is the process of teaching computers to make decisions without being explicitly programmed. It includes supervised, unsupervised, and reinforcement learning approaches.""",

  """Retrieval-Augmented Generation (RAG) combines a retrieval system with a generative model. It improves accuracy by retrieving relevant documents from a knowledge base before generating answers.""",

  """LangChain is a framework designed to build applications powered by large language models. It provides abstractions for chains, agents, and memory, making it easier to integrate LLMs with external data.""",

  """When using RAG with LangChain, the pipeline typically involves embedding documents, storing them in a vector database, retrieving relevant chunks, and passing them into the LLM for context-aware generation.""",

  """Popular vector databases used with LangChain for RAG pipelines include Pinecone, Weaviate, FAISS, and Milvus.""",

  """Deep learning models often require GPUs for efficient training because of their high computational requirements.""",

  """LangChain supports integration with multiple LLM providers, including OpenAI, Anthropic, and Hugging Face.""",

  """In a RAG pipeline, embeddings are numerical vector representations of text that allow semantic similarity search.""",

  """Fine-tuning machine learning models involves adjusting model parameters on domain-specific datasets to improve accuracy."""
]


### Document Loading
- But as I am using a dummy data over here, we can skip this part , or store it in a folder data and use directoryloader to import them

In [3]:
from langchain.docstore.document import Document
documents = [Document(page_content=doc) for doc in sample_docs] 
documents 

[Document(metadata={}, page_content='Deep learning is a subfield of machine learning that uses neural networks with multiple layers. It is widely applied in image recognition, natural language processing, and speech recognition.'),
 Document(metadata={}, page_content='Machine learning is the process of teaching computers to make decisions without being explicitly programmed. It includes supervised, unsupervised, and reinforcement learning approaches.'),
 Document(metadata={}, page_content='Retrieval-Augmented Generation (RAG) combines a retrieval system with a generative model. It improves accuracy by retrieving relevant documents from a knowledge base before generating answers.'),
 Document(metadata={}, page_content='LangChain is a framework designed to build applications powered by large language models. It provides abstractions for chains, agents, and memory, making it easier to integrate LLMs with external data.'),
 Document(metadata={}, page_content='When using RAG with LangChain,

### Document Splitting

In [5]:
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    separators=[" "],
    chunk_size=100,
    chunk_overlap=10,
    length_function=len
)
chunks = text_splitter.split_documents(documents)
print(f"Number of chunks: {len(chunks)}")
print(chunks[4])  

Number of chunks: 23
page_content='Retrieval-Augmented Generation (RAG) combines a retrieval system with a generative model. It'


### Embeddings using HuggingFace

In [6]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2") 
vector = embeddings.embed_query("Hello world")
print(vector)  

c:\Users\rites\Desktop\RAG\RAG\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[-0.03447727486491203, 0.03102317824959755, 0.006734970025718212, 0.026108985766768456, -0.03936202451586723, -0.16030244529247284, 0.06692401319742203, -0.006441489793360233, -0.0474504791200161, 0.014758856035768986, 0.07087527960538864, 0.05552763119339943, 0.019193334504961967, -0.026251312345266342, -0.01010954286903143, -0.02694045566022396, 0.022307461127638817, -0.022226648405194283, -0.14969263970851898, -0.017493007704615593, 0.00767625542357564, 0.05435224249958992, 0.0032543970737606287, 0.031725890934467316, -0.0846213847398758, -0.02940601296722889, 0.05159561336040497, 0.04812406003475189, -0.0033148222137242556, -0.058279167860746384, 0.04196927323937416, 0.022210685536265373, 0.1281888335943222, -0.022338971495628357, -0.011656315997242928, 0.06292839348316193, -0.032876335084438324, -0.09122604131698608, -0.031175347045063972, 0.0526994913816452, 0.04703482985496521, -0.08420311659574509, -0.030056199058890343, -0.02074483036994934, 0.009517835453152657, -0.0037217906

### Initalize ChromaDB

In [11]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

persist_dir = "./chroma_db"

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model,
    collection_name="rag_collection",
    persist_directory=persist_dir,
)
vector_db.persist()

print(f"Number of documents in the collection: {vector_db._collection.count()}")


C:\Users\rites\AppData\Local\Temp\ipykernel_22828\2883594600.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


Number of documents in the collection: 23


C:\Users\rites\AppData\Local\Temp\ipykernel_22828\2883594600.py:14: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_db.persist()


### Testing Similarity Search

In [18]:
query = "What is Fine tuning?"
result = vector_db.similarity_search(query,k=3)
result

[Document(metadata={}, page_content='Fine-tuning machine learning models involves adjusting model parameters on domain-specific datasets'),
 Document(metadata={}, page_content='datasets to improve accuracy.'),
 Document(metadata={}, page_content='programmed. It includes supervised, unsupervised, and reinforcement learning approaches.')]

### Advance Similarity Search with Scores
- Lower the value closer to the result

In [ ]:
results = vector_db.similarity_search_with_score(query,k=3)
results

[(Document(metadata={}, page_content='Fine-tuning machine learning models involves adjusting model parameters on domain-specific datasets'),
  1.1072428226470947),
 (Document(metadata={}, page_content='datasets to improve accuracy.'),
  1.422195315361023),
 (Document(metadata={}, page_content='programmed. It includes supervised, unsupervised, and reinforcement learning approaches.'),
  1.5049494504928589)]